In [9]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig

def extract_sample_number(file_name):
    match = re.search(r'sample #(\d+)', file_name)
    if match:
        return int(match.group(1))
    # Return a large number to push files without sample numbers to the end
    return float('inf')

def data_SDR(path_folder):
    # List all files in the folder
    file_list = os.listdir(path_folder)

    # Sort files by sample number
    sorted_file_list = sorted(file_list, key=extract_sample_number)

    # Initialize data matrix based on the sorted file list
    exploration_path = os.path.join(path_folder, sorted_file_list[0])
    data_exploration = np.load(exploration_path)
    
    # Check if the data is complex
    is_complex = np.iscomplexobj(data_exploration)
    
    # Initialize the data matrix with the correct dtype
    data_matrix = np.zeros((len(sorted_file_list), data_exploration.shape[0]), dtype=complex if is_complex else float)

    # Load all files into data_matrix
    for pos, file_name in enumerate(sorted_file_list):
        full_path = os.path.join(path_folder, file_name)
        print(f'file {full_path} Done.')
        data = np.load(full_path)
        #print(f"Loading {file_name} (sample #{extract_sample_number(file_name)}), shape = {data.shape}")
        data_matrix[pos] = data

    print("Data matrix loading complete.")
    return data_matrix

def basic_threshold(Pxx_window, percentage_threshold):
        threshold = np.mean(Pxx_window)
        
        while True:
            points_below_threshold = np.sum(Pxx_window < threshold)
            total_points = len(Pxx_window)

            percentage_below = points_below_threshold / total_points

            if percentage_below < percentage_threshold:
                break

            threshold *= 0.95
        return threshold

In [10]:
folder_path = r'C:\Samples-tdt-Hack-RF'

tdt_matrix = data_SDR(folder_path)
M, N = tdt_matrix.shape
print(f'Matrix shape= ({M},{N})')
for i in range(tdt_matrix.shape[0]):
    print(f'Shape tdt_{i+1}= {tdt_matrix[i].shape}')

    
fs_bw = 20e6  # Frecuencia de muestreo (ajustar a la frecuencia de muestreo de tu SDR)
nperseg = N - 1
psd_matrix = np.zeros((M,(nperseg//2) + 1))
for i in range(M):
    freq, Pxx = sig.welch(tdt_matrix[i], fs=fs_bw, nperseg=1024, scaling='density', return_onesided=False)
    psd_matrix[i] = Pxx

file C:\Samples-tdt-Hack-RF\tdt.npy Done.
file C:\Samples-tdt-Hack-RF\tdt_2.npy Done.
file C:\Samples-tdt-Hack-RF\tdt_3.npy Done.
Data matrix loading complete.
Matrix shape= (3,20000000)
Shape tdt_1= (20000000,)
Shape tdt_2= (20000000,)
Shape tdt_3= (20000000,)


ValueError: could not broadcast input array from shape (1024,) into shape (10000000,)